In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization, LayerNormalization, Bidirectional, Attention
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Load data
data1 = pd.read_csv("personalized_social_scheduler_10k.csv")
data=data1.head(500)
# Encode categorical variables
label_encoders = {}
categorical_cols = ["Category", "Interaction_Type", "Day_of_Week", "Preferred_Time_Slot", "Best_Time_to_Meet"]

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode target variable (Friend_ID)
friend_encoder = LabelEncoder()
data["Friend_ID"] = friend_encoder.fit_transform(data["Friend_ID"])
label_encoders["Friend_ID"] = friend_encoder

# Normalize numerical columns
scaler = MinMaxScaler()
numerical_cols = ["Duration_Minutes", "Mood_Before", "Mood_After", "Previous_Interaction_Gap", "Interaction_Frequency_Score", "Social_Preference_Score", "Urgency_Score"]
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Data augmentation: Adding Gaussian noise to numerical features
def augment_data(df, noise_level=0.02):
    augmented_df = df.copy()
    noise = np.random.normal(0, noise_level, df[numerical_cols].shape)
    augmented_df[numerical_cols] += noise
    return augmented_df

augmented_data = augment_data(data)
data = pd.concat([data, augmented_data], axis=0).reset_index(drop=True)

# Prepare dataset for LSTM
sequence_length = 10
features = numerical_cols + categorical_cols
target_friend = "Friend_ID"
target_time = "Best_Time_to_Meet"

X, y_friend, y_time = [], [], []
for i in range(len(data) - sequence_length):
    X.append(data[features].iloc[i:i+sequence_length].values)
    y_friend.append(data[target_friend].iloc[i + sequence_length])
    y_time.append(data[target_time].iloc[i + sequence_length])

X = np.array(X)
y_friend = to_categorical(np.array(y_friend))
y_time = to_categorical(np.array(y_time))

# Train-test split
X_train, X_test, y_friend_train, y_friend_test, y_time_train, y_time_test = train_test_split(
    X, y_friend, y_time, test_size=0.2, random_state=42
)

# Attention Layer
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],), initializer="zeros", trainable=True)
        self.v = self.add_weight(shape=(input_shape[-1], 1), initializer="random_normal", trainable=True)

    def call(self, inputs):
        score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.v, axes=1), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

# Build improved LSTM model
input_layer = Input(shape=(sequence_length, X.shape[2]))

x = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.01)))(input_layer)
x = LayerNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = LayerNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Attention Mechanism
attention_output = AttentionLayer()(x)

x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(attention_output)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)

# Output layers
friend_output = Dense(y_friend.shape[1], activation='softmax', name='friend_output')(x)
time_output = Dense(y_time.shape[1], activation='softmax', name='time_output')(x)

# Define model
model = Model(inputs=input_layer, outputs={'friend_output': friend_output, 'time_output': time_output})

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005, amsgrad=True),
    loss={'friend_output': 'categorical_crossentropy', 'time_output': 'categorical_crossentropy'},
    metrics={'friend_output': 'accuracy', 'time_output': 'accuracy'}
)

# Learning Rate Scheduling and Early Stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
model.fit(
    X_train, {'friend_output': y_friend_train, 'time_output': y_time_train},
    epochs=200, batch_size=64, validation_split=0.2,
    callbacks=[reduce_lr]
)

# Generate a new sample input for testing
sample_input = np.random.rand(1, sequence_length, X.shape[2])
predicted_output = model.predict(sample_input)
predicted_friend = np.argmax(predicted_output['friend_output'])
predicted_time = np.argmax(predicted_output['time_output'])

# Decode predictions
friend_decoded = friend_encoder.inverse_transform([predicted_friend])[0]
time_decoded = label_encoders["Best_Time_to_Meet"].inverse_transform([predicted_time])[0]

print(f"Predicted Friend to Interact With: {friend_decoded}")
print(f"Predicted Best Time to Interact: {time_decoded}")

# Save model
model.save("social_scheduler_v3.h5")

print("Model trained, evaluated, and saved successfully.")


<ipython-input-4-d7cc42a9450b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-4-d7cc42a9450b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-4-d7cc42a9450b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 24s 543ms/step - friend_output_accuracy: 0.0174 - friend_output_loss: 3.9890 - loss: 25.3567 - time_output_accuracy: 0.2547 - time_output_loss: 1.5033 - val_friend_output_accuracy: 0.0377 - val_friend_output_loss: 3.8790 - val_loss: 24.3784 - val_time_output_accuracy: 0.2579 - val_time_output_loss: 1.4034 - learning_rate: 5.0000e-04
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - friend_output_accuracy: 0.0206 - friend_output_loss: 3.8789 - loss: 24.1738 - time_output_accuracy: 0.2625 - time_output_loss: 1.4501 - val_friend_output_accuracy: 0.0126 - val_friend_output_loss: 3.8748 - val_loss: 23.3546 - val_time_output_accuracy: 0.2579 - val_time_output_loss: 1.4026 - learning_rate: 5.0000e-04
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 237ms/step - friend_output_accuracy: 0.0218 - friend_output_loss: 3.8867 - loss: 23.1221 - time_output_accuracy: 0.2861 - time_output_loss: 1.4057 - val_friend_output_accuracy: 0.0314 - val_friend_output_loss: 3

Predicted Friend to Interact With: Work_16
Predicted Best Time to Interact: Afternoon
Model trained, evaluated, and saved successfully.


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, BatchNormalization, LayerNormalization, Bidirectional, Attention
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split

# Load data
data1 = pd.read_csv("personalized_social_scheduler_10k.csv")
data=data1.head(500)
# Encode categorical variables
label_encoders = {}
categorical_cols = ["Category", "Interaction_Type", "Day_of_Week", "Preferred_Time_Slot", "Best_Time_to_Meet"]

for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode target variable (Friend_ID)
friend_encoder = LabelEncoder()
data["Friend_ID"] = friend_encoder.fit_transform(data["Friend_ID"])
label_encoders["Friend_ID"] = friend_encoder

# Normalize numerical columns
scaler = MinMaxScaler()
numerical_cols = ["Duration_Minutes", "Mood_Before", "Mood_After", "Previous_Interaction_Gap", "Interaction_Frequency_Score", "Social_Preference_Score", "Urgency_Score"]
data[numerical_cols] = scaler.fit_transform(data[numerical_cols])

# Data augmentation: Adding Gaussian noise to numerical features
def augment_data(df, noise_level=0.02):
    augmented_df = df.copy()
    noise = np.random.normal(0, noise_level, df[numerical_cols].shape)
    augmented_df[numerical_cols] += noise
    return augmented_df

augmented_data = augment_data(data)
data = pd.concat([data, augmented_data], axis=0).reset_index(drop=True)

# Prepare dataset for LSTM
sequence_length = 10
features = numerical_cols + categorical_cols
target_friend = "Friend_ID"
target_time = "Best_Time_to_Meet"

X, y_friend, y_time = [], [], []
for i in range(len(data) - sequence_length):
    X.append(data[features].iloc[i:i+sequence_length].values)
    y_friend.append(data[target_friend].iloc[i + sequence_length])
    y_time.append(data[target_time].iloc[i + sequence_length])

X = np.array(X)
y_friend = to_categorical(np.array(y_friend))
y_time = to_categorical(np.array(y_time))

# Train-test split
X_train, X_test, y_friend_train, y_friend_test, y_time_train, y_time_test = train_test_split(
    X, y_friend, y_time, test_size=0.2, random_state=42
)

# Attention Layer
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self):
        super(AttentionLayer, self).__init__()

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]), initializer="random_normal", trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],), initializer="zeros", trainable=True)
        self.v = self.add_weight(shape=(input_shape[-1], 1), initializer="random_normal", trainable=True)

    def call(self, inputs):
        score = tf.nn.tanh(tf.tensordot(inputs, self.W, axes=1) + self.b)
        attention_weights = tf.nn.softmax(tf.tensordot(score, self.v, axes=1), axis=1)
        context_vector = attention_weights * inputs
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector

# Build improved LSTM model
input_layer = Input(shape=(sequence_length, X.shape[2]))

x = Bidirectional(LSTM(512, return_sequences=True, kernel_regularizer=l2(0.01)))(input_layer)
x = LayerNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(256, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = LayerNormalization()(x)
x = Dropout(0.3)(x)

x = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01)))(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

# Attention Mechanism
attention_output = AttentionLayer()(x)

x = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(attention_output)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)

# Output layers
friend_output = Dense(y_friend.shape[1], activation='softmax', name='friend_output')(x)
time_output = Dense(y_time.shape[1], activation='softmax', name='time_output')(x)

# Define model
model = Model(inputs=input_layer, outputs={'friend_output': friend_output, 'time_output': time_output})

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005, amsgrad=True),
    loss={'friend_output': 'categorical_crossentropy', 'time_output': 'categorical_crossentropy'},
    metrics={'friend_output': 'accuracy', 'time_output': 'accuracy'}
)

# Learning Rate Scheduling and Early Stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.0001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train model
model.fit(
    X_train, {'friend_output': y_friend_train, 'time_output': y_time_train},
    epochs=200, batch_size=64, validation_split=0.2,
    callbacks=[reduce_lr]
)

# Generate a new sample input for testing
sample_input = np.random.rand(1, sequence_length, X.shape[2])
predicted_output = model.predict(sample_input)
predicted_friend = np.argmax(predicted_output['friend_output'])
predicted_time = np.argmax(predicted_output['time_output'])

# Decode predictions
friend_decoded = friend_encoder.inverse_transform([predicted_friend])[0]
time_decoded = label_encoders["Best_Time_to_Meet"].inverse_transform([predicted_time])[0]

print(f"Predicted Friend to Interact With: {friend_decoded}")
print(f"Predicted Best Time to Interact: {time_decoded}")

# Save model
model.save("social_scheduler_v3.h5")

print("Model trained, evaluated, and saved successfully.")




<ipython-input-2-538561336a41>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-2-538561336a41>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[col] = le.fit_transform(data[col])
<ipython-input-2-538561336a41>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 38s 1s/step - friend_output_accuracy: 0.0126 - friend_output_loss: 3.9601 - loss: 44.5425 - time_output_accuracy: 0.2828 - time_output_loss: 1.4804 - val_friend_output_accuracy: 0.0126 - val_friend_output_loss: 3.8888 - val_loss: 42.3050 - val_time_output_accuracy: 0.2830 - val_time_output_loss: 1.3835 - learning_rate: 5.0000e-04
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 831ms/step - friend_output_accuracy: 0.0259 - friend_output_loss: 3.9402 - loss: 41.7206 - time_output_accuracy: 0.2706 - time_output_loss: 1.4297 - val_friend_output_accuracy: 0.0189 - val_friend_output_loss: 3.8767 - val_loss: 39.5058 - val_time_output_accuracy: 0.2893 - val_time_output_loss: 1.3818 - learning_rate: 5.0000e-04
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 834ms/step - friend_output_accuracy: 0.0285 - friend_output_loss: 3.9069 - loss: 38.8582 - time_output_accuracy: 0.2978 - time_output_loss: 1.3796 - val_friend_output_accuracy: 0.0377 - val_friend_output_loss: 3.8

Predicted Friend to Interact With: Work_13
Predicted Best Time to Interact: Afternoon
Model trained, evaluated, and saved successfully.


NameError: name 'joblib' is not defined

In [3]:
import joblib

# Save label encoders
joblib.dump(label_encoders, "label_encoders.pkl")

# Save friend encoder separately
joblib.dump(friend_encoder, "friend_encoder.pkl")

# Save MinMaxScaler
joblib.dump(scaler, "scaler.pkl")

# Save feature columns list
joblib.dump(features, "features.pkl")

print("Preprocessing objects saved successfully.")

Preprocessing objects saved successfully.
